### 학습 모델 Inference

In [ ]:
import os
import torch
from dataclasses import dataclass
import pandas as pd
from transformers import (
    WhisperForConditionalGeneration, WhisperProcessor
)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
model_id = "openai/whisper-tiny"
checkpoint_dir = f"/workspace/results/whisper_train/{model_id.split('/')[-1]}/checkpoint-10000"
lang = "ko"
task = "transcribe"

test_df = pd.read_csv("/workspace/kru_data/test.csv")
sample_file = test_df.iloc[0]["abs_path"]
# sample_file = "/workspace/kor_med_stt_data/snu_data/말이좀늦는것같아요_샘플데이터셋/남학생의사_여환자_1.wav"

In [ ]:
import librosa
waveform, sr = librosa.load(sample_file, sr=16000)
# 처음 5초는 넘어가고 5~35초 가지고오기
start_sec = 0
end_sec = 3
start_sample = start_sec * sr
end_sample = end_sec * sr
if len(waveform) > end_sample:
    print(f"This file length is {len(waveform)/sr} seconds. Using audio from {start_sec} to {end_sec} seconds.")
    waveform = waveform[start_sample:end_sample]
else:
    print(f"This file length is {len(waveform)/sr} seconds. Using available audio from {start_sec} seconds to end.")
    waveform = waveform[start_sample:]
    
# sampling된 오디오 jupyter 에서 듣기
import IPython.display as ipd
ipd.Audio(waveform, rate=sr)

In [ ]:
processor = WhisperProcessor.from_pretrained(
    model_id, language="ko", task="transcribe"
)
# model = WhisperForConditionalGeneration.from_pretrained(checkpoint_dir).to("cuda")
model = WhisperForConditionalGeneration.from_pretrained(model_id).to("cuda")

In [ ]:
def transcribe_long_audio(processor, model, waveform, sr, chunk_sec=5):
    chunk_size = chunk_sec * sr
    texts = []
    last_text = ""

    for start in range(0, len(waveform), chunk_size):
        end = start + chunk_size
        chunk = waveform[start:end]
        if len(chunk) == 0:
            continue

        # Whisper inference
        input_features = processor.feature_extractor(
            chunk,
            sampling_rate=sr,
            return_tensors="pt"
        ).input_features.to("cuda")

        with torch.no_grad():
            pred_ids = model.generate(input_features)

        text = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)[0]

        # 🔥 중복 제거
        if last_text:
            # 마지막 15자 기준으로 중복 검사
            overlap = last_text[-15:]
            if text.startswith(overlap):
                text = text[len(overlap):]

        texts.append(text)
        last_text = text

    return " ".join(texts).strip()

full_text = transcribe_long_audio(
    processor,
    model,
    waveform,
    sr,
    chunk_sec=3,     # 여기서 원하는 길이 설정
)

print(full_text)

## 모델 평가

In [ ]:
from utils.eval import fast_asr_metrics
import json 
import pandas as pd

# omniasr inference model
# file_path = "/workspace/results/omniasr_inference/omniasr_ctc/omniASR_CTC_300M/test_pred.parquet" # omni asr 

# whisper train model 
file_path = "/workspace/results/whisper_test/whisper-tiny/checkpoint-10000/test_pred.parquet" # whisper train model
# whisper inference model
# file_path = "/workspace/results/whisper_inference/whisper_tiny_inference/test_pred.parquet" # whisper inference model

save_path = file_path.replace(file_path.split("/")[-1], "metrics.json") # file_path 의 디렉토리
df = pd.read_parquet(file_path)

metrics = fast_asr_metrics(df, gt_col = "gt_text", pred_col = "pred_text")
print(metrics)

with open(save_path, "w") as f:
    json.dump(metrics, f, indent=4)
    print(f"Metrics saved to {save_path}")

### 전체 평가

In [1]:
from utils.eval import fast_asr_metrics
import json 
import pandas as pd
from glob import glob 
# test_root_dir = "/workspace/results/"
test_root_dir = "/workspace/results/omniasr_inference/omniasr_ctc"

# test_root_dir 내 모든 test_pred.parquet 파일을 재귀적으로 찾기
test_files = sorted(glob(f"{test_root_dir}/**/test_pred.parquet", recursive=True))
all_list = []
for file_path in test_files:
    save_path = file_path.replace(file_path.split("/")[-1], "metrics.df") # file_path 의 디렉토리
    df = pd.read_parquet(file_path)

    metrics = fast_asr_metrics(df, gt_col = "gt_text", pred_col = "pred_text")
    metrics['model_name'] = file_path.split("/")[-2]
    print(file_path.split("/")[-2])
    print(metrics)

    # metrics를 데이터프레임으로 변환 후 csv로 저장
    
    df_metrics = pd.DataFrame([metrics])
    df_metrics = df_metrics.sort_values(by='wer_mean', ascending=True)
    df_metrics.to_csv(save_path, index=False)
    print(f"Metrics dataframe saved to {save_path}")
    
    all_list.append(metrics)

all_df = pd.DataFrame(all_list)
# 'model_name' 컬럼이 가장 첫 번째 컬럼으로 오도록 순서 조정
cols = ['model_name'] + [col for col in all_df.columns if col != 'model_name']
all_df = all_df[cols]
all_df = all_df.sort_values(by='wer_mean', ascending=True)
all_df.to_csv(f"{test_root_dir}/all_metrics.csv", index=False)
all_df.round(3)

omniASR_CTC_1B
{'wer_all': 54.651519604453355, 'cer_all': 18.163502070508773, 'wer_mean': 59.63098968856431, 'wer_std': 29.609255000260532, 'wer_median': 57.14285714285714, 'wer_min': 0.0, 'wer_max': 1600.0, 'cer_mean': 20.513082126528033, 'cer_std': 20.685031205686414, 'cer_median': 16.0, 'cer_min': 0.0, 'cer_max': 2750.0, 'inference_time_sec_mean': 0.18741014499440212, 'inference_time_sec_std': 0.06796365556821761, 'inference_time_sec_median': 0.1851564683020115, 'inference_time_sec_min': 0.0, 'inference_time_sec_max': 1.2473665475845337, 'num_samples': 254490, 'model_name': 'omniASR_CTC_1B'}
Metrics dataframe saved to /workspace/results/omniasr_inference/omniasr_ctc/omniASR_CTC_1B/metrics.df
omniASR_CTC_300M
{'wer_all': 64.72840744070258, 'cer_all': 23.660343479633273, 'wer_mean': 69.22924869355926, 'wer_std': 28.80134183342591, 'wer_median': 66.66666666666666, 'wer_min': 0.0, 'wer_max': 1400.0, 'cer_mean': 26.57213591647686, 'cer_std': 25.709417460134752, 'cer_median': 20.512820512

,model_name,wer_all,cer_all,wer_mean,wer_std,wer_median,wer_min,wer_max,cer_mean,cer_std,cer_median,cer_min,cer_max,inference_time_sec_mean,inference_time_sec_std,inference_time_sec_median,inference_time_sec_min,inference_time_sec_max,num_samples
3,omniASR_CTC_7B,51.239,17.134,56.609,30.625,50.000,0.0,1500.0,19.957,23.730,14.286,0.0,2700.0,0.745,0.260,0.691,0.0,5.925,254449
2,omniASR_CTC_3B,51.951,17.570,57.483,30.922,50.000,0.0,1700.0,20.461,23.579,14.815,0.0,2800.0,0.456,0.154,0.440,0.0,2.798,254490
0,omniASR_CTC_1B,54.652,18.164,59.631,29.609,57.143,0.0,1600.0,20.513,20.685,16.000,0.0,2750.0,0.187,0.068,0.185,0.0,1.247,254490
1,omniASR_CTC_300M,64.728,23.660,69.229,28.801,66.667,0.0,1400.0,26.572,25.709,20.513,0.0,2700.0,0.073,0.047,0.085,0.0,0.555,254490


In [ ]:
import datetime

def hh_mm_ss_to_seconds(time_str):
    # Split off microseconds if present
    if '.' in time_str:
        time_str, micro = time_str.split('.')
    else:
        micro = '0'
    t = datetime.datetime.strptime(time_str, "%H:%M:%S")
    delta = datetime.timedelta(
        hours=t.hour, minutes=t.minute, seconds=t.second, microseconds=int(micro)
    )
    return delta.total_seconds()

def second_to_hh_mm_ss(seconds):
    return datetime.timedelta(seconds=seconds)

second = hh_mm_ss_to_seconds("10:49:38.484706")
print(second)

hh_mm_ss = second_to_hh_mm_ss(second * 1 / 5)
print(f"{hh_mm_ss}")


In [ ]:
pd.read_parquet("/workspace/results/omniasr_inference/omniasr_ctc/omniASR_CTC_1B/test_pred.parquet")